# Import ML library, initialize GPBoost model

In [13]:
import pandas as pd, pickle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import gpboost
import gpboost as gpb

class GPBoostClassifier:
    def __init__(self,scan=False,use_coords=False,random_state=42,boosting_type='gbdt'):
        self.scan=scan
        self.use_coords=use_coords
        self.random_state=random_state
        self.boosting_type=boosting_type
    
    def fit(self,X,y,groups,coords=None):
        data_train = gpb.Dataset(X, y)
        self.gp_model = gpb.GPModel(group_data=groups, likelihood="bernoulli_logit", gp_coords=coords.values if self.use_coords else None,cov_function="exponential")
        params = {'learning_rate': 1e1, 'min_data_in_leaf': 20, 'objective': "binary",
                  'verbose': 0}
        if self.boosting_type!='gbdt':
            assert self.boosting_type in ['rf','dart']
            params['boosting']=self.boosting_type
        params['n_jobs']=1
        num_boost_round = 600
        
        if self.scan:
            param_grid_small = {'learning_rate': [0.1,0.01,0.001], 'min_data_in_leaf': [20,50,100],
                                'max_depth': [5,10,15], 'max_bin': [255,1000], 'use_nesterov_acc': [False,True]}
            opt_params = gpb.grid_search_tune_parameters(param_grid=param_grid_small,
                                                         params=params,
                                                         num_try_random=15,
                                                         folds=list(GroupShuffleSplit(random_state=42).split(X,y,groups)),
                                                         gp_model=self.gp_model,
                                                         use_gp_model_for_validation=True,
                                                         train_set=data_train,
                                                         verbose_eval=1,
                                                         num_boost_round=num_boost_round,#50 
                                                         early_stopping_rounds=10,
                                                         seed=1,
                                                         metrics='binary_logloss') 

            params=opt_params['best_params']

        self.gpm = gpb.train(params=params,
                    train_set=data_train,
                    gp_model=self.gp_model,
                    num_boost_round=num_boost_round,
                    
                   )
        return self
    
    def predict_proba(self,X,groups,coords=None):
        y_pred = self.gpm.predict(data=X, group_data_pred=groups, gp_coords_pred=coords.values if self.use_coords else None,
                            predict_var=True, raw_score=False)['response_mean']
        return y_pred

# Load data

In [14]:
import pyreadr
from sklearn.model_selection import GroupShuffleSplit,StratifiedShuffleSplit
import pandas as pd
covar=['age','sex','MLH1']
expr=pd.read_pickle("../../data/dsp_data_igg_expr.pkl")
pheno=pd.read_pickle("../../data/dsp_data_igg_pheno.pkl")
pheno['sex']=(pheno['sex']=="M").astype(int)
for k in pheno['macro_annot'].unique():
    pheno[f'macro_{k}']=(pheno['macro_annot']==k).astype(int)

# Fit models by each architecture, extract interactions using SHAP and save to file to read into R

In [39]:
import numpy as np
import tqdm
import shap
from sklearn.metrics import roc_auc_score
from functools import reduce
from kneed import KneeLocator

def fit_model(expr,pheno,macro,outcome):
    use_macro=(macro=="overall")
    if not use_macro:
        expr=expr[pheno['macro_annot']==macro]
        pheno=pheno[pheno['macro_annot']==macro]
    if outcome=="ln_only":
        expr=expr[pheno['Distant_Mets']==0]
        pheno=pheno[pheno['Distant_Mets']==0]
    elif outcome=="Distant_Mets":
        expr=expr[pheno['ln_only']==0]
        pheno=pheno[pheno['ln_only']==0]
    covar=['age','sex','MLH1']+([] if not use_macro else ['macro_inter','macro_intra'])
    ss=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=None,
                    train_size=0.7)
    data_dict=[]
    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=expr,y=pheno[outcome],groups=pheno['batch']))):
        data_dict_=dict()
        data_dict_['expr_train'],data_dict_['pheno_train']=expr.iloc[train_idx],pheno.iloc[train_idx]
        data_dict_['expr_test'],data_dict_['pheno_test']=expr.iloc[test_idx],pheno.iloc[test_idx]
        data_dict.append(data_dict_)

    results=[]
    mods=[]
    for cv_idx in tqdm.trange(len(data_dict)):
        np.random.seed(42)
        data_dict_=data_dict[i].copy()
        X_train,X_test,y_train,y_test,g_train,g_test=pd.concat([data_dict_['expr_train'],data_dict_['pheno_train'][covar]],axis=1),\
                                                        pd.concat([data_dict_['expr_test'],data_dict_['pheno_test'][covar]],axis=1),\
                                                        data_dict_['pheno_train'][outcome],\
                                                        data_dict_['pheno_test'][outcome],\
                                                        data_dict_['pheno_train']['batch'],\
                                                        data_dict_['pheno_test']['batch']
        gpc=GPBoostClassifier(random_state=42).fit(X_train,y_train,g_train)
        y_pred=gpc.predict_proba(X_test,y_test,g_test)
        results.append(dict(y_pred=y_pred,
                              y_true=y_test.astype(float),
                               data_dict=data_dict_))#,mod=gpc
        mods.append(gpc)

    pickle.dump(results,open(f"./analyses/4_MEML/pickle_res/gpbres-{macro}-{outcome}.pkl",'wb'))
    print(np.mean([roc_auc_score(d_['y_true'],d_['y_pred']) for d_ in results]))
    
    explainers=[]
    shap_vals=[]
    shap_interactions=[]

    for cv_idx in tqdm.trange(len(data_dict)):
        gpc=mods[cv_idx]
        explainers.append(shap.TreeExplainer(gpc.gpm))
        data_dict_=results[cv_idx]['data_dict'].copy()
        X_train=pd.concat([data_dict_['expr_train'],data_dict_['pheno_train'][covar]],axis=1)
        shap_vals.append(explainers[-1].shap_values(X_train,data_dict_['pheno_train']['batch'],check_additivity=False))
        shap_interactions.append(explainers[-1].shap_interaction_values(X_train,data_dict_['pheno_train']['batch']))
    pickle.dump(dict(shap_vals=shap_vals,
                shap_interactions=shap_interactions),open(f"./analyses/4_MEML/pickle_res/shap_interactions-{macro}-{outcome}.pkl",'wb'))

    index_col=expr.columns.tolist()+covar
    df_interaction=pd.DataFrame(np.abs(np.concatenate(shap_interactions,0)).mean(0),index=index_col,columns=index_col)
    df_interaction[np.eye(*df_interaction.shape).astype(bool)]=0
    all_interaction_shap_scores=reduce(lambda x,y:x+y,[df_interaction]).where(np.triu(np.ones(df_interaction.shape),k=1).astype(np.bool)).stack().reset_index()
    all_interaction_shap_scores.columns=['feature_1','feature_2', 'shap_interaction_score']
    all_interaction_shap_scores=all_interaction_shap_scores.sort_values('shap_interaction_score',ascending=False)
    kneed=KneeLocator(np.arange(all_interaction_shap_scores.shape[0]), all_interaction_shap_scores['shap_interaction_score'], direction='decreasing', curve='convex',
                      S=150.0)
    n_top_interactions=min(600,kneed.knee)
    form_str=['+'.join(index_col),'+'.join(map(lambda x:f"{x[0]}:{x[1]}",all_interaction_shap_scores.iloc[:n_top_interactions].iloc[:,:2].values.tolist()))]
    pd.to_pickle(form_str,f"./analyses/4_MEML/pickle_res/form_str-{macro}-{outcome}.pkl")
    return 1

In [41]:
import dask
from dask.diagnostics import ProgressBar
with ProgressBar():
    finished=dask.compute(*[ dask.delayed(fit_model)(expr,pheno,macro,outcome) for outcome in ['any_mets','ln_only','Distant_Mets'] for macro in ['overall','inter','intra','away']],scheduler="processes")
        

[########################################] | 100% Completed | 10min 13.0s
